In [144]:
#this project 

In [145]:
import requests 
import sys
import json as j 
import pandas as pd
from IPython import display

In [146]:
###########################
#        Entities         #
###########################

class ResponsePokemonList:
    def __init__(self, name):
        self.name = name

class ResponseEvolutionChainList:
    def __init__(self, url):
        self.url = url

class Pokemon: 
    def __init__(self, id, name, base_experience, height, hp, attack, defense, special_attack, special_defense, speed):
        self.id = id
        self.name = name
        self.base_experience = base_experience
        self.height = height
        self.hp = hp
        self.attack = attack
        self.defense = defense  
        self.special_attack = special_attack
        self.special_defense = special_defense
        self.speed = speed

class TypePokemon: 
    def __init__(self, id, typeName):
        self.id = id
        self.typeName = typeName

class AbilityPokemon: 
    def __init__(self, id, abilityName):
        self.id = id
        self.abilityName = abilityName

class MoveDetail: 
    def __init__(self, id, move, learnMethod, gameVersion):
        self.id = id
        self.move = move
        self.learnMethod = learnMethod
        self.gameVersion = gameVersion

class EvolutionChain:
    def __init__(self, id, firstForm, secondForm, thirdForm):
        self.id = id
        self.firstForm = firstForm
        self.secondForm = secondForm
        self.thirdForm = thirdForm

In [147]:
"""
this funcion is responsible to test if the response is valid 
params: 
    - res: API response 
return response serialized as json 
"""
def handleResponse(res):
    if res == None:
        raise Exception("Response is empty!") 
    json_response = j.loads(res.text)   
    return json_response  

In [148]:
"""
this function is responsible to obtain the list of pokemons names avaible in API. 
The names will be used to make the requests to fetch the pokemons attributes. 
return a list of ResponsePokemonList class
"""
def getPokemonNameList():
    list_pokemonResponse = []
    list_pokemonName = []

    API_POKENAMES = 'https://pokeapi.co/api/v2/pokemon/?offset=0&limit='+str(sys.maxsize)
    #API_POKENAMES = 'https://pokeapi.co/api/v2/pokemon/?offset=0&limit=10'

    #request API and validate the response 
    try:
        res = requests.get(API_POKENAMES)
        list_pokemonResponse = handleResponse(res)
    except(requests.exceptions.RequestException, Exception) as err:
        print("Request error! Error:",err)
        raise SystemExit(err)

    #create the object response to populate the pokemon's names list 
    try:
        for i in list_pokemonResponse['results']:
            name_pokemon = ResponsePokemonList(i['name'])
            list_pokemonName.append(name_pokemon)
    except(Exception) as err:
        print("Error while populate list. Error: ", err)

    return list_pokemonName

In [149]:
"""
this function is responsible to obtain the pokemon evolution chain avaible in API. 
The functin will store the request links of every evolution chain. 
return a list of ResponseEvolutionChainList
"""
def getEvoChainURLList():

    evoChain_URL = []
    evoChainURL_list = []
    API_POKEEVOURL = 'https://pokeapi.co/api/v2/evolution-chain?offset=0&limit='+str(sys.maxsize)

    #request API and validate the response 
    try:
        res = requests.get(API_POKEEVOURL)
        evoURL_pokemonResponse = handleResponse(res)
    except(requests.exceptions.RequestException, Exception) as err:
        print("Request error! Error:",err)
        raise SystemError(err)

    try:
        for i in evoURL_pokemonResponse['results']:
            evoChain_URL = ResponseEvolutionChainList(i['url'])
            evoChainURL_list.append(evoChain_URL)
    except(Exception) as err:
        print("Error while populate list. Error: ", err)

    return evoChainURL_list


In [150]:
#get the name of pokemons 
pokeName_list = getPokemonNameList()

#lists for create the dataframes 
pokedex = []
types = []
abilities = []
gameVersions = []
moves = []

for pokeName in pokeName_list: 

    API_POKEATTRIBUTES = 'https://pokeapi.co/api/v2/pokemon/'+pokeName.name

    #request API and validate the response 
    try:
        res = requests.get(API_POKEATTRIBUTES)
        attributes_pokemonResponse = handleResponse(res)
    except(requests.exceptions.RequestException, Exception) as err:
        print("Request error! Error:",err)
        raise SystemError(err)

    pokemon_id = attributes_pokemonResponse['id']

    try:

        #populate the pokemons types list 
        for i in attributes_pokemonResponse['types']:
            types.append(TypePokemon(pokemon_id, i['type']['name']))

        #populate the pokemons ability list
        for i in attributes_pokemonResponse['abilities']:
            abilities.append(AbilityPokemon(pokemon_id, i['ability']['name']))
        
        #populate the pokemons moves list
        for i in attributes_pokemonResponse['moves']: 
            for x in i['version_group_details']: 
                moves.append(MoveDetail(pokemon_id, i['move']['name'], x['move_learn_method']['name'], x['version_group']['name']))
        
        #populate the pokemons stats
        hp = 0 
        attack = 0 
        defense = 0
        special_attack = 0 
        special_defense = 0
        speed = 0
        for i in attributes_pokemonResponse['stats']:
                if i['stat']['name'] == 'hp':
                    hp = i['base_stat']

                if i['stat']['name'] == 'attack':
                    attack = i['base_stat']
                
                if i['stat']['name'] == 'defense':
                    defense = i['base_stat']
                
                if i['stat']['name'] == 'special-attack':
                    special_attack = i['base_stat']
                
                if i['stat']['name'] == 'special-defense':
                    special_defense = i['base_stat']
                
                if i['stat']['name'] == 'speed':
                    speed = i['base_stat']
                


        #populate the pokemons list
        pokedex.append(Pokemon(
                        attributes_pokemonResponse['id'], 
                        attributes_pokemonResponse['name'], 
                        attributes_pokemonResponse['base_experience'], 
                        attributes_pokemonResponse['height'], 
                        hp, 
                        attack, 
                        defense, 
                        special_attack, 
                        special_defense, 
                        speed
                        )
                    )
                    
    except(Exception) as err:
        print("Err", err)

    # print(pokedex)
    # print(types)
    # print(abilities)
    # print(gameVersions)
    # print(moves)

In [151]:
#get the evolution chain URL 
evoChainURL_list = getEvoChainURLList()

evoChain_list = []

for evochain in evoChainURL_list:

    #request API and validate the response 
    try:
        res = requests.get(evochain.url)
        evoChain_pokemonResponse = handleResponse(res)
    except(requests.exceptions.RequestException, Exception) as err:
        print("Request error! Error:",err)
        raise SystemError(err)
    
    evoChain_id = evoChain_pokemonResponse['id']
    firstForm = evoChain_pokemonResponse['chain']['species']['name'] 
    
    try:
        for i in evoChain_pokemonResponse['chain']['evolves_to']:
            secondForm = i['species']['name']
            
            for x in i['evolves_to']:
                thirdForm = x['species']['name']
                evoChain_list.append(EvolutionChain(evoChain_id, firstForm, secondForm, thirdForm))

    except(Exception) as err:
        print("Error while populate list. Error: ", err)

In [152]:
###########################
#  create the dataframes  #
###########################

pokedex_df = pd.DataFrame([x.__dict__ for x in pokedex])
types_df = pd.DataFrame([x.__dict__ for x in types])
abilities_df = pd.DataFrame([x.__dict__ for x in abilities])
moves_df = pd.DataFrame([x.__dict__ for x in moves])

evolutionChain_df = pd.DataFrame(x.__dict__ for x in evoChain_list)

In [153]:
#Higher Pokemon
higherPoke_df = pokedex_df[['name', 'height']].sort_values('height', ascending=False).head(5) 
print('Most higher pokemon:')
display.display(higherPoke_df)

Most higher pokemon:


,name,height
1087,eternatus-eternamax,1000
1117,centiskorch-gmax,750
1122,duraludon-gmax,430
1112,coalossal-gmax,420
1108,inteleon-gmax,400


In [154]:
#Pokemons with more than one evolution path
moreThanOneEvo = evolutionChain_df[evolutionChain_df['id'].duplicated(keep = False)]
countPoke = moreThanOneEvo['firstForm'].nunique()

print('Number of Pokemons with more than one evolution path: '+ str(countPoke))
display.display(moreThanOneEvo)


Number of Pokemons with more than one evolution path: 5


,id,firstForm,secondForm,thirdForm
12,18,oddish,gloom,vileplume
13,18,oddish,gloom,bellossom
14,26,poliwag,poliwhirl,poliwrath
15,26,poliwag,poliwhirl,politoed
43,135,wurmple,silcoon,beautifly
44,135,wurmple,cascoon,dustox
47,140,ralts,kirlia,gardevoir
48,140,ralts,kirlia,gallade
98,413,cosmog,cosmoem,solgaleo
99,413,cosmog,cosmoem,lunala


In [155]:
#Pokemon that offers the most experience when being defeated

#merge pokemon types with pokedex to get the base_experience 
pokeTypes_df = types_df.merge(pokedex_df[['id', 'name', 'base_experience']], left_on='id', right_on='id')

#merge the table above with evolution chain to get only the pokemons in the first form 
pokeTypesFirstForm_df = pokeTypes_df.merge(evolutionChain_df[['firstForm']], left_on='name', right_on='firstForm')

#only the ice pokemons 
icePoke_df = pokeTypesFirstForm_df.loc[pokeTypesFirstForm_df['typeName']=='ice']

#select only the columns I want and sort the base experience
icePoke_df = icePoke_df[['name', 'base_experience']].sort_values('base_experience', ascending=False).head(5)

print('pokemon that offers the most experience when being defeated: ')
display.display(icePoke_df)



pokemon that offers the most experience when being defeated: 


,name,base_experience
107,vanillite,61
72,spheal,58
45,swinub,50


In [156]:
#the moves that is most learned by the level-up method, what version of the game does this happen
#from the Pokemon that learn the moves, the Pokemon that have the highest stat value attack.
pokeMoves_df = moves_df.merge(pokedex_df[['id', 'name', 'attack']], left_on='id', right_on='id')

pokeMovesFirstForm_df = pokeMoves_df.merge(evolutionChain_df[['firstForm']], left_on='name', right_on='firstForm')

mostLvlUp = pokeMovesFirstForm_df.loc[pokeMovesFirstForm_df['learnMethod']=='level-up'].groupby(['move', 'learnMethod', 'gameVersion']).size()

mostMoveLvlUp_df = pd.DataFrame(mostLvlUp).reset_index()
mostMoveLvlUp_df = mostMoveLvlUp_df.rename(columns={0:'freqMoveLvlUp'})
mostMoveLvlUp_df = mostMoveLvlUp_df.sort_values('freqMoveLvlUp', ascending=False).head(1)

pokeMostMoveLvlUp_df = mostMoveLvlUp_df.merge(pokeMoves_df[['name', 'move', 'attack']], left_on='move', right_on='move').drop_duplicates()

pokeAttack = pokeMostMoveLvlUp_df[['name', 'gameVersion', 'attack']].sort_values('attack', ascending=False).head(10)

print('the move that is most learned by the level-up method and what version of the game does this happen:')
display.display(mostMoveLvlUp_df)

print('among the Pokemon that learn the moves, the Pokemon that have the highest stat value attack: ')
display.display(pokeAttack)

the move that is most learned by the level-up method and what version of the game does this happen:


,move,learnMethod,gameVersion,freqMoveLvlUp
4318,tackle,level-up,ultra-sun-ultra-moon,36


among the Pokemon that learn the moves, the Pokemon that have the highest stat value attack: 


,name,gameVersion,attack
2785,heracross-mega,ultra-sun-ultra-moon,185
2797,garchomp-mega,ultra-sun-ultra-moon,170
2897,calyrex-ice,ultra-sun-ultra-moon,165
2819,diancie-mega,ultra-sun-ultra-moon,160
2894,darmanitan-galar-zen,ultra-sun-ultra-moon,160
2805,swampert-mega,ultra-sun-ultra-moon,150
2694,glastrier,ultra-sun-ultra-moon,145
2770,aegislash-blade,ultra-sun-ultra-moon,140
2789,aggron-mega,ultra-sun-ultra-moon,140
2893,darmanitan-galar-standard,ultra-sun-ultra-moon,140


In [157]:
#Pokemons with the highest stat boost
firstFormAttributes = evolutionChain_df.merge(pokedex_df[['name','hp','attack','defense','special_attack','special_defense','speed']], left_on='firstForm',right_on='name')
secondFormAttributes = firstFormAttributes.merge(pokedex_df[['name','hp','attack','defense','special_attack','special_defense','speed']], left_on='secondForm',right_on='name')

diff_hp = secondFormAttributes['hp_y'] - secondFormAttributes['hp_x']  
diff_attack = secondFormAttributes['attack_y'] - secondFormAttributes['attack_x']  
diff_defense = secondFormAttributes['defense_y'] - secondFormAttributes['defense_x']  
diff_special_attack = secondFormAttributes['special_attack_y'] - secondFormAttributes['special_attack_x']  
diff_special_defense = secondFormAttributes['special_defense_y'] - secondFormAttributes['special_defense_x'] 
diff_speed = secondFormAttributes['speed_y'] - secondFormAttributes['speed_x'] 
sum_diff = diff_hp + diff_attack + diff_defense + diff_special_attack + diff_special_defense + diff_speed

secondFormAttributes['diff_hp'] = diff_hp
secondFormAttributes['diff_attack'] = diff_attack
secondFormAttributes['diff_defense'] = diff_defense
secondFormAttributes['diff_special_attack'] = diff_special_attack 
secondFormAttributes['diff_special_defense'] = diff_special_defense
secondFormAttributes['diff_speed'] = diff_speed
secondFormAttributes['sum_diff'] = sum_diff

biggestStatBoost = secondFormAttributes[['firstForm','secondForm','diff_hp','diff_attack','diff_defense','diff_special_attack','diff_special_defense','diff_speed','sum_diff']].drop_duplicates().sort_values('sum_diff', ascending=False).head(7)

print('7 Pokemons with the highest stat boost from first form to second form: ')
display.display(biggestStatBoost)

Pokemons with the highest stat boost from first form to second form: 


,firstForm,secondForm,diff_hp,diff_attack,diff_defense,diff_special_attack,diff_special_defense,diff_speed,sum_diff
23,happiny,chansey,150,0,0,20,40,20,230
11,zubat,golbat,35,35,35,35,35,35,210
37,swinub,piloswine,50,50,40,30,30,0,200
98,cosmog,cosmoem,0,0,100,0,100,0,200
42,zigzagoon,linoone,40,40,20,20,20,40,180
105,rolycoly,carkol,50,20,40,20,20,20,170
49,slakoth,vigoroth,20,20,20,20,20,60,160


In [158]:
#Group types with more attack advantage

pokeTypes_df = types_df.merge(pokedex_df[['id', 'name', 'attack']], left_on='id', right_on='id')

typesFirstForm_df = pokeTypes_df.merge(evolutionChain_df[['firstForm']], left_on='name', right_on='firstForm').drop_duplicates()

y = []
df_list = typesFirstForm_df.values.tolist()
for i in df_list:
    next = df_list.index(i) + 1
    if(next > len(df_list)-1):
        break
    else:
        next_item = df_list[next]
    
    if(i[0] == next_item[0]):
        y.append([i[0],i[2],i[1]+"/"+next_item[1]])
    else:
        previus_item = df_list[next-2]
        if i[0]!=previus_item[0]:
            y.append([i[0],i[2],i[1]])
        
AttackAdvantage_df = pd.DataFrame(y, columns=['id','name','typeGroup'])

AttackAdvantage_df = AttackAdvantage_df.merge(typesFirstForm_df[['id','attack']], left_on='id', right_on='id').drop_duplicates()

AttackAdvantage_df = AttackAdvantage_df.groupby(['typeGroup']).sum()

AttackAdvantage_df = AttackAdvantage_df.sort_values('attack', ascending=False).head(20).drop("id", axis=1)

print("Group types with more attack advantage: ")
display.display(AttackAdvantage_df)

Group types with more attack advantage: 


,attack
typeGroup,
water,579
fire,541
grass,403
dragon,331
normal/flying,280
normal,266
electric,263
grass/poison,204
bug,192


Index(['attack'], dtype='object')
